In [1]:
%sh timeout 1 nc 54.213.33.240 9002

In [2]:
from pyspark.sql.functions import *
from pyspark.sql.streaming import *
from pyspark.sql.types import *
from datetime import datetime

In [3]:
%sql SET spark.sql.shuffle.partitions = 3 

In [4]:
lines = spark.readStream \
    .format("socket") \
    .option("host", "54.213.33.240") \
    .option("port", 9002) \
    .load()

In [5]:
edits = lines.select(json_tuple('value', "channel", "timestamp", "isRobot", "isAnonymous")) \
  .selectExpr("c0 as channel", "cast(c1 as timestamp) as time", "c2 as page") \
  .createOrReplaceTempView("edits")

In [6]:
parquetData = sql("select * from edits")
display(parquetData)


In [7]:
editCounts = spark.sql("""SELECT count(*), channel, date_format(window(time, '10 seconds').start, 'HH:mm:ss') as time 
                              FROM edits 
                              GROUP BY channel, window(time, '10 seconds')
                              ORDER BY time""")

In [8]:
display(editCounts)